In [ ]:
!git clone --branch master_colab https://github.com/aravindadithya/DLR

In [ ]:
!pip install visdom
#python -m visdom.server -port 8097

In [1]:
import sys
import os
current_dir = os.getcwd()
print(current_dir)

parent_dir = os.path.join(current_dir, 'DLR')
model_dir = os.path.join(parent_dir, 'trained_models', 'MNIST', 'model4', 'nn_models/')
'''
parent_dir='C:\\Users\\garav\\AGOP\\DLR'
model_dir= 'C:\\Users\\garav\\AGOP\\DLR\\trained_models\\MNIST\\model4\\nn_models\\'
'''
#parent_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
sys.path.append(parent_dir)

In [3]:
import torch
import torchvision
import torchvision.transforms as transforms
from utils import agop_gcnn as agc
from utils import trainer as tr
from torch.utils.data import Dataset
import random
import torch.backends.cudnn as cudnn
import rfm
import numpy as np
from trained_models.MNIST.model4 import trainer as t
import numpy as np
from sklearn.model_selection import train_test_split
from torch.linalg import norm
from torchvision import models
import torch.optim as optim
import torch.nn.functional as F
import torch.nn as nn
from utils.groupy.gconv.pytorch_gconv.splitgconv2d import P4ConvZ2, P4ConvP4, P4MConvZ2, P4MConvP4M
from groupy.gconv.make_gconv_indices import *
from copy import deepcopy
from torch.nn.functional import pad
from torch.func import jacrev

Setting up a new session...
Without the incoming socket you cannot receive events from the server or register event handlers to your Visdom client.


In [5]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
#device='cpu'
print(f"Using device: {device}")


Using device: cuda:0


In [ ]:
import torch_xla.core.xla_model as xm
device = xm.xla_device()
print(device)

In [7]:
torch.cuda.empty_cache()

In [9]:
trainloader, valloader, testloader = t.get_loaders()
net= t.get_untrained_net()
init_net= deepcopy(net)
import os
if os.path.exists(model_dir+'mnist_gcnn_trained_nn.pth'):
    checkpoint = torch.load(model_dir+'mnist_gcnn_trained_nn.pth', map_location=torch.device(device))
    net.load_state_dict(checkpoint['state_dict'])  # Access the 'state_dict' within the loaded dictionary
    print("Model weights loaded successfully.")

#print("Train the network first")
#t.train_net()

Model weights loaded successfully.


In [11]:
tr.visualize_predictions(net, testloader, range(10), device, num_images=36)

C:\Users\garav\AGOP\DLR\trained_models\MNIST\model4\model4.py:44: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


In [13]:
agc.verify_NFA(net, init_net, trainloader, layer_idx=3)

l_idx 7
Computing GOP for sample 0 out of 10
Computing GOP for sample 1 out of 10
Computing GOP for sample 2 out of 10
Computing GOP for sample 3 out of 10
Computing GOP for sample 4 out of 10
Computing GOP for sample 5 out of 10
Computing GOP for sample 6 out of 10
Computing GOP for sample 7 out of 10
Computing GOP for sample 8 out of 10
Computing GOP for sample 9 out of 10
Computing GOP for sample 10 out of 10
Shape after gradients:  torch.Size([720, 720])
Correlation between Trained Gcnn and AGOP:  tensor(0.9653, device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward0>)
Correlation between Un-trained Gcnn and AGOP:  tensor(0.0986, device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward0>)


tensor([[ 0.8657,  0.0083,  0.2049,  ..., -0.1020, -0.0330,  0.0882],
        [ 0.0083,  0.8676,  0.0457,  ..., -0.1038,  0.3065,  0.1029],
        [ 0.2049,  0.0457,  0.8468,  ...,  0.0284,  0.0456,  0.1190],
        ...,
        [-0.1020, -0.1038,  0.0284,  ...,  0.9005,  0.0070, -0.0297],
        [-0.0330,  0.3065,  0.0456,  ...,  0.0070,  0.7943, -0.0055],
        [ 0.0882,  0.1029,  0.1190,  ..., -0.0297, -0.0055,  0.7218]],
       dtype=torch.float64)

In [15]:
agc.verify_NFA(net, init_net, trainloader, layer_idx=2)

l_idx 5
Computing GOP for sample 0 out of 10
Computing GOP for sample 1 out of 10
Computing GOP for sample 2 out of 10
Computing GOP for sample 3 out of 10
Computing GOP for sample 4 out of 10
Computing GOP for sample 5 out of 10
Computing GOP for sample 6 out of 10
Computing GOP for sample 7 out of 10
Computing GOP for sample 8 out of 10
Computing GOP for sample 9 out of 10
Computing GOP for sample 10 out of 10
Shape after gradients:  torch.Size([360, 360])
Correlation between Trained Gcnn and AGOP:  tensor(0.9334, device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward0>)
Correlation between Un-trained Gcnn and AGOP:  tensor(0.1745, device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward0>)


tensor([[ 1.4325,  0.1340, -0.2771,  ..., -0.0348, -0.1890,  0.1651],
        [ 0.1340,  1.2878, -0.2561,  ...,  0.2383, -0.0286,  0.0565],
        [-0.2771, -0.2561,  1.2542,  ..., -0.1439,  0.0051, -0.0365],
        ...,
        [-0.0348,  0.2383, -0.1439,  ...,  1.5052,  0.0078, -0.0170],
        [-0.1890, -0.0286,  0.0051,  ...,  0.0078,  1.3672,  0.2067],
        [ 0.1651,  0.0565, -0.0365,  ..., -0.0170,  0.2067,  1.4088]],
       dtype=torch.float64)

In [17]:
agc.verify_NFA(net, init_net, trainloader, layer_idx=1)

l_idx 2
Computing GOP for sample 0 out of 10
Computing GOP for sample 1 out of 10
Computing GOP for sample 2 out of 10
Computing GOP for sample 3 out of 10
Computing GOP for sample 4 out of 10
Computing GOP for sample 5 out of 10
Computing GOP for sample 6 out of 10
Computing GOP for sample 7 out of 10
Computing GOP for sample 8 out of 10
Computing GOP for sample 9 out of 10
Computing GOP for sample 10 out of 10
Shape after gradients:  torch.Size([360, 360])
Correlation between Trained Gcnn and AGOP:  tensor(0.9770, device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward0>)
Correlation between Un-trained Gcnn and AGOP:  tensor(0.0836, device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward0>)


tensor([[ 1.5122,  0.5156,  0.1316,  ..., -0.3701,  0.1405, -0.4765],
        [ 0.5156,  1.7492,  0.1980,  ...,  0.2609,  0.0725, -0.2701],
        [ 0.1316,  0.1980,  1.8413,  ..., -0.6061, -0.3391, -0.5612],
        ...,
        [-0.3701,  0.2609, -0.6061,  ...,  3.3044,  0.7460, -0.0309],
        [ 0.1405,  0.0725, -0.3391,  ...,  0.7460,  2.1470,  0.8858],
        [-0.4765, -0.2701, -0.5612,  ..., -0.0309,  0.8858,  2.7916]],
       dtype=torch.float64)

In [19]:
agc.verify_NFA(net, init_net, trainloader, layer_idx=0)

l_idx 0
Computing GOP for sample 0 out of 10
Computing GOP for sample 1 out of 10
Computing GOP for sample 2 out of 10
Computing GOP for sample 3 out of 10
Computing GOP for sample 4 out of 10
Computing GOP for sample 5 out of 10
Computing GOP for sample 6 out of 10
Computing GOP for sample 7 out of 10
Computing GOP for sample 8 out of 10
Computing GOP for sample 9 out of 10
Computing GOP for sample 10 out of 10
Shape after gradients:  torch.Size([9, 9])
Correlation between Trained Gcnn and AGOP:  tensor(0.9710, device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward0>)
Correlation between Un-trained Gcnn and AGOP:  tensor(0.7090, device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward0>)


tensor([[ 32.8832,  11.0137,  -0.2061,  14.9455,   9.8338,  -8.2748,  -0.8288,
          -7.7451, -13.8880],
        [ 11.0137,  40.9705,  15.7780,  -2.2490,  10.3861,  -1.0840, -10.1144,
         -11.8915,  -7.0418],
        [ -0.2061,  15.7780,  34.8157,  -8.3311,  10.1940,  12.0150, -15.8977,
          -9.9664,  -0.3288],
        [ 14.9455,  -2.2490,  -8.3311,  40.5436,  10.5808, -11.4767,  12.1232,
          -1.3768,  -9.2086],
        [  9.8338,  10.3861,  10.1940,  10.5808,  61.7055,  10.3711,  10.2591,
          10.8842,  10.0160],
        [ -8.2748,  -1.0840,  12.0150, -11.4767,  10.3711,  40.4735,  -8.5745,
          -2.9043,  14.5416],
        [ -0.8288, -10.1144, -15.8977,  12.1232,  10.2591,  -8.5745,  35.3570,
          16.6435,  -0.5595],
        [ -7.7451, -11.8915,  -9.9664,  -1.3768,  10.8842,  -2.9043,  16.6435,
          42.0940,  11.3312],
        [-13.8880,  -7.0418,  -0.3288,  -9.2086,  10.0160,  14.5416,  -0.5595,
          11.3312,  32.8482]], dtype=torch.float6

# Testing and Debug Section:

In [19]:
def trans_filter(w, inds):
    inds_reshape = inds.reshape((-1, inds.shape[-1])).astype(np.int64)
    w_indexed = w[:, :, inds_reshape[:, 0].tolist(), inds_reshape[:, 1].tolist(), inds_reshape[:, 2].tolist()]
    w_indexed = w_indexed.view(w_indexed.size()[0], w_indexed.size()[1],
                                    inds.shape[0], inds.shape[1], inds.shape[2], inds.shape[3])
    w_transformed = w_indexed.permute(0, 2, 1, 3, 4, 5)
    return w_transformed.contiguous()
'''
layer_idx=2
count=0
# Get the layer_idx+1 th conv layer
for idx, m in enumerate(net.children()):
        if isinstance(m, P4ConvZ2):
            print("hi")
            count += 1
        if count-1 == layer_idx:
            l_idx = idx
      break
            '''
l_idx =7
layer = deepcopy(net.features[l_idx])
print(l_idx)
print(layer.weight)
print(layer.kernel_size)
print(layer.padding)
print(layer.stride)
# Extract W matrix
tw = trans_filter(layer.weight, layer.inds)
tw_shape = (layer.out_channels * layer.output_stabilizer_size,
                    layer.in_channels * layer.input_stabilizer_size,
                    layer.ksize, layer.ksize)
M = tw.view(tw_shape)


#_, ki, q, s = M.shape
#print(M.shape)
#_,ki, q, s = M.shape

k, ki, q,s= M.shape
            
# Build W which is a (k, c*q*s) matrix. What to do with ip_stab
M = M.reshape(-1, ki*q*s)
            
# Compute WtW which is (c*q*s,c*q*s) matrix
M = torch.einsum('nd, nD -> dD', M, M)


print(M)




7
Parameter containing:
tensor([[[[[-1.3622e-02,  3.3417e-02, -5.3618e-02],
           [ 1.4870e-02, -6.1207e-02,  4.2695e-02],
           [-3.5703e-02, -6.2889e-02, -7.3429e-02]],

          [[ 3.8961e-02, -3.5383e-02,  1.0853e-02],
           [ 7.3087e-02, -1.0702e-02,  9.9781e-03],
           [ 1.0437e-02,  6.6061e-02,  8.2599e-02]],

          [[-3.8585e-02, -5.6168e-02, -6.6992e-02],
           [-3.0202e-02,  4.1048e-02, -1.1283e-01],
           [-4.5873e-02, -5.4089e-02, -8.5588e-02]],

          [[-1.9788e-03, -5.6791e-02, -5.7679e-02],
           [ 3.6108e-02, -7.0742e-02,  3.3539e-02],
           [ 6.6735e-02,  6.4161e-02, -7.1869e-02]]],


         [[[-3.7501e-02, -6.2126e-02, -3.6880e-02],
           [-1.6727e-02, -1.7260e-02, -1.5512e-02],
           [-5.8266e-03,  2.8563e-02, -8.0181e-03]],

          [[ 2.1066e-02, -5.1789e-02,  6.5678e-02],
           [ 1.2947e-02, -1.5812e-02,  6.3621e-02],
           [ 2.2949e-02, -2.8164e-02,  6.0984e-02]],

          [[ 3.4684e-02,  

In [21]:
def correlation(M1, M2):
    M1 -= M1.mean()
    M2 -= M2.mean()

    norm1 = norm(M1.flatten())
    norm2 = norm(M2.flatten())

    return torch.sum(M1.cuda() * M2.cuda()) / (norm1 * norm2)

def patchify(x, in_channels, ip_stab, patch_size, stride_size, padding=None, pad_type='zeros'):
    '''
        Given an input image (n,c,h,w) generate (n,w_out,h_out,c,q,s) respecting stride,padding, 
        w_out is number of pathces along the width for the given stride after padding
        h_out is number of pathces along the height for the given stride after padding
        (q,s) is the kernel dimensions 
    '''
    input_shape = x.size()
    #TODO: The last two shapes look swapped. This was the same order in cohens code too. For square ips 
    #there is no effect. However for rect ips what would happen?
    x = x.view(input_shape[0], in_channels*ip_stab, input_shape[-2], input_shape[-1])
    #x = x.view(input_shape[0], in_channels*ip_stab, input_shape[-2], input_shape[-1])
    q1, q2 = patch_size
    s1, s2 = stride_size
    print("Image Shape",x.shape)
    if padding is None:
        pad_1 = (q1-1)//2
        pad_2 = (q2-1)//2
    else:
        pad_1, pad_2 = padding

    pad_dims = (pad_2, pad_2, pad_1, pad_1)
    if pad_type == 'zeros':
        x = pad(x, pad_dims)
    elif pad_type == 'circular':
        x = pad(x, pad_dims, 'circular')
        
    patches = x.unfold(2, q1, s1).unfold(3, q2, s2) #(n, c, h_out, w_out, q, s)
    print("Image Shape1",patches.shape)
    patches = patches.transpose(1, 3).transpose(1, 2) #(n,w_out,h_out,c,q,s) 
    print("Image Shape2",patches.shape)
    return patches

def trans_filter(w, inds):
    inds_reshape = inds.reshape((-1, inds.shape[-1])).astype(np.int64)
    w_indexed = w[:, :, inds_reshape[:, 0].tolist(), inds_reshape[:, 1].tolist(), inds_reshape[:, 2].tolist()]
    w_indexed = w_indexed.view(w_indexed.size()[0], w_indexed.size()[1],
                                    inds.shape[0], inds.shape[1], inds.shape[2], inds.shape[3])
    w_transformed = w_indexed.permute(0, 2, 1, 3, 4, 5)
    return w_transformed.contiguous()

class PatchConvLayer(nn.Module):
    def __init__(self, conv_layer):
        super().__init__()
        self.layer = conv_layer #(k,c,q,s)
        #inds = make_c4_z2_indices(self.layer.ksize)
       
    def forward(self, patches):
        tw = trans_filter(self.layer.weight, self.layer.inds)
        tw_shape = (self.layer.out_channels * self.layer.output_stabilizer_size,
                    self.layer.in_channels * self.layer.input_stabilizer_size,
                    self.layer.ksize, self.layer.ksize)
        tw = tw.view(tw_shape)
        print("tw shape",tw.shape)
        print("Patch_shape", patches.shape)
        #n, _, ny_out, nx_out = patches.shape()
        #print(tw)
        out = torch.einsum('nhwcqr, kcqr -> nhwk', patches, tw)
        n, w, h, k = out.shape
        out = out.transpose(1, 3).transpose(2, 3) #(n,k,h_out,w_out)
        out = out.view(n, self.layer.out_channels, self.layer.output_stabilizer_size, h, w)
        print("out_shape", out.shape)
        return out



for idx, batch in enumerate(trainloader):
    imgs,_=batch
    break

img=imgs[6:7]
l_idx=7
img=img.to(device)
img = img.double()
#layers= list(net.children())
#print("Shape of wight matrix:",layers[l_idx].weight.shape)
net=net.double()
net = net.to(device)
img = net.features[:l_idx](img)
print("Image shape",img.shape)



patchnet = deepcopy(net)
temp= deepcopy(net.features[l_idx])
layer = PatchConvLayer(temp)
patchnet.features = deepcopy(net.features[l_idx:])
patchnet.features[0] = layer
patchnet.to(device)
#op = patchnet(patchify(img, temp.kernel_size, temp.stride, temp.padding))
op1 = patchnet.features[0](patchify(img, temp.in_channels, 
                                    temp.input_stabilizer_size, temp.kernel_size, temp.stride, temp.padding))
print(op1.shape)
print(op1)
#print(patchnet2)


patchnet2 = deepcopy(net)
layer = deepcopy(net.features[l_idx])
patchnet2.features = deepcopy(net.features[l_idx:])
patchnet2.features[0] = layer
patchnet2.features[0].bias = None
patchnet2.features[0].padding = (0,0)
patchnet2.to(device)
#print(patchnet2)
#op2= patchnet2(img)
op2= patchnet2.features[0](img)
#print(op2.shape)
#print(op2)
print(torch.allclose(op1,op2,rtol=0.00000000001,atol=0, equal_nan=False))

def single_net(x):
            patchnet.eval()
            return patchnet(x).squeeze(0)

def single_net2(x):
            patchnet2.eval()
            return patchnet2(x).squeeze(0)

op=jacrev(single_net)(patchify(img, temp.in_channels, 
                                    temp.input_stabilizer_size, temp.kernel_size, temp.stride, temp.padding))
n, c, w, h, _, _, _ = op.shape
patches = op.transpose(1, 3).transpose(1, 2) #(n, h_out, w_out, chunk, c, q, s)
grads = patches.reshape(n*w*h, c, -1) #(n*w_out*h_out, chunk, c*q*s)
#print("Grads:",grads.shape)
#Clarify: Where is mean taken
ajop = torch.einsum('ncd, ncD -> dD', grads, grads) #(c*q*s,c*q*s)
ajop= sqrt(ajop)
print(correlation(ajop, M))


Image shape torch.Size([1, 20, 4, 10, 10])
Image Shape torch.Size([1, 80, 10, 10])
Image Shape1 torch.Size([1, 80, 8, 8, 3, 3])
Image Shape2 torch.Size([1, 8, 8, 80, 3, 3])
tw shape torch.Size([80, 80, 3, 3])
Patch_shape torch.Size([1, 8, 8, 80, 3, 3])
out_shape torch.Size([1, 20, 4, 8, 8])
torch.Size([1, 20, 4, 8, 8])
tensor([[[[[-2.8604e+01, -3.2986e+01, -1.4823e+01,  ..., -2.6281e+01,
            -7.6946e+00,  5.0898e-01],
           [-3.2669e+01, -2.8716e+01, -4.9976e+00,  ..., -2.4996e+01,
            -1.8147e+01,  2.2979e+00],
           [-3.1694e+01, -3.1319e+01, -1.4660e+01,  ..., -3.2587e+01,
            -2.3655e+01, -2.9430e+00],
           ...,
           [-1.7325e+01, -1.8296e+01, -2.9455e+01,  ..., -1.5495e+01,
            -7.0917e-01, -4.7588e-01],
           [-1.3635e+01, -7.0738e+00, -1.0939e+01,  ..., -3.2141e+01,
            -9.3699e+00, -2.2551e+00],
           [-8.1362e+00, -1.0422e+01, -2.3231e+01,  ..., -3.7060e+01,
            -1.0718e+01, -3.3273e+00]],

       

In [104]:
def co(M1):
    M= M1.clone()
    M -= M.mean()
    #M.flatten()
    return M
M= torch.randn(3, 3)
print(M)
co(M)
print(M)

tensor([[-0.1223, -1.2220,  1.0906],
        [ 0.2609, -1.7309, -1.0689],
        [-1.8210,  1.9365, -1.7288]])
tensor([[-0.1223, -1.2220,  1.0906],
        [ 0.2609, -1.7309, -1.0689],
        [-1.8210,  1.9365, -1.7288]])
